# **Movie Predictions Part 2B**

**Name:** **Derek Overton**

**Date:** **3/1/2023**

**Porject: Movie Perdictions**

# **Import Data**

In [1]:
# Install tmdbsimple (only need to run once)
!pip install tmdbsimple

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gzip

# Additional Imports
import os, json, math, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [3]:
with open('C:/Users/Beastmode225/.secret/tmbd_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [4]:
tmdb.API_KEY =  login['api-key']

## **Define Function**

In [5]:
def write_json(new_data, filename):
    """Appends a list of records (new_data) to a json file (filename).
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""

    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [6]:
def get_movie_with_rating(movie_id):
    """ Adapted from source github/celuao/tmdbsimple"""
    #Get the movie object for the current id
    movie = tmdb.Movies(movie_id)

    # Save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()

    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation == US
        if c['iso_3166_1'] == 'US':
            # save a 'certification' key in info with the certification
            info['certification'] = c['certification']

    return info

In [7]:
# Load JSON file into pandas dataframe
with open('Data/tmdb_api_results_2000.json', 'r') as file:
    data = json.load(file)

df = pd.json_normalize(data)

# Save dataframe as compressed CSV file
with gzip.open('Data/tmdb_api_results_2000.csv.gz', 'wt', encoding='utf8') as file:
    df.to_csv(file, index=False)

In [8]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/basics.csv.gz')

In [9]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008,NaN,94,Horror
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [10]:
ratings = pd.read_csv('Data/ratings.csv.gz')

In [11]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2083
1,tt0000002,5.6,281
2,tt0000005,6.2,2820
3,tt0000006,5.0,194
4,tt0000007,5.4,886


In [12]:
df = pd.read_csv('Data/final_tmdb_data_2000.csv.gz')
df.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.500,22.0,NaN
2,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,5.100,8.0,NaN
3,tt0116391,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,0.0,152.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0.0,4.000,1.0,NaN
4,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,843.0,cn,花樣年華,...,12854953.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.111,2143.0,PG


In [13]:
#create empty dataframe
df = pd.DataFrame()
#Load files to the empty dataframe
for i in range(2000,2021):
   f = f'Data/final_tmdb_data_{i}.csv.gz'
   df = pd.concat([df,pd.read_csv(f)])

df.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.500,22.0,NaN
2,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,5.100,8.0,NaN
3,tt0116391,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,0.0,152.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0.0,4.000,1.0,NaN
4,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,843.0,cn,花樣年華,...,12854953.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.111,2143.0,PG


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28117 entries, 0 to 3891
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                28117 non-null  object 
 1   adult                  28096 non-null  float64
 2   backdrop_path          20021 non-null  object 
 3   belongs_to_collection  1719 non-null   object 
 4   budget                 28096 non-null  float64
 5   genres                 28096 non-null  object 
 6   homepage               7131 non-null   object 
 7   id                     28096 non-null  float64
 8   original_language      28096 non-null  object 
 9   original_title         28096 non-null  object 
 10  overview               27521 non-null  object 
 11  popularity             28096 non-null  float64
 12  poster_path            26951 non-null  object 
 13  production_companies   28096 non-null  object 
 14  production_countries   28096 non-null  object 
 15  release_

## **Create Required Lists for the Loop**

In [15]:
YEARS_TO_GET = [2000,2001]

### **define an errors list**

In [16]:
errors = [ ]

In [17]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['akas.csv.gz',
 'basics.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'final_tmdb_data_2002.csv.gz',
 'final_tmdb_data_2003.csv.gz',
 'final_tmdb_data_2004.csv.gz',
 'final_tmdb_data_2005.csv.gz',
 'final_tmdb_data_2006.csv.gz',
 'final_tmdb_data_2007.csv.gz',
 'final_tmdb_data_2008.csv.gz',
 'final_tmdb_data_2009.csv.gz',
 'final_tmdb_data_2010.csv.gz',
 'final_tmdb_data_2011.csv.gz',
 'final_tmdb_data_2012.csv.gz',
 'final_tmdb_data_2013.csv.gz',
 'final_tmdb_data_2014.csv.gz',
 'final_tmdb_data_2015.csv.gz',
 'final_tmdb_data_2016.csv.gz',
 'final_tmdb_data_2017.csv.gz',
 'final_tmdb_data_2018.csv.gz',
 'final_tmdb_data_2019.csv.gz',
 'final_tmdb_data_2020.csv.gz',
 'ratings.csv.gz',
 'tmbd_api.json',
 'tmdb_api_results_2000.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2002.json',
 'tmdb_api_results_2003.json',
 'tmdb_api_results_2004.json',
 'tmdb_api_results_2005.json',
 'tmdb_api_results_2006.json',
 'tmdb_

In [18]:
['title_akas.csv.gz','title_basics.csv.gz','title_ratings.csv.gz']

['title_akas.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

## **Start INNER Loop**

### **Interate through the list of Movie IDs and make the calls**

In [19]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    df = basics.loc[basics['startYear']==YEAR].copy()
    movie_ids = df['tconst'].copy()
    previous_df = pd.read_json(JSON_FILE)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    # Iterating over movie ID
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            temp = get_movie_with_rating(movie_id)
            write_json(temp,JSON_FILE)
            time.sleep(0.02)

        except Exception as e:
            errors.append([movie_id, e])
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/287 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/333 [00:00<?, ?it/s]

In [20]:
print(f"- Total errors: {len(errors)}")

- Total errors: 404


In [21]:
print(df.columns)

Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')


# **Exploratory Data Analysis** 

In [22]:
df1 = pd.read_json('Data/tmdb_api_results_2000.json')

In [23]:
df2 = pd.read_json('Data/tmdb_api_results_2001.json')

In [24]:
data = pd.concat([df1, df2], ignore_index=True, sort=False)

In [25]:
data.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification,origin_country
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,None,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",,62127.0,en,The Fantasticks,...,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.500,22.0,,NaN
2,tt0113092,0.0,None,None,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",,110977.0,en,For the Cause,...,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,5.100,8.0,,NaN
3,tt0116391,0.0,None,None,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",,442869.0,hi,Gang,...,152.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,,Gang,0.0,4.000,1.0,NaN,NaN
4,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,None,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",,843.0,cn,花樣年華,...,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.111,2143.0,PG,NaN


## **How many movies had at least some valid financial information (values > 0 for budget OR revenue)?**

In [26]:
# Filter out any movies with 0's for budget AND revenue
valid_movies = data[(data['budget'] > 0) | (data['revenue'] > 0)]

# Count the number of movies with valid financial information
num_valid_movies = len(valid_movies)

print(f'The number of movies with valid financial information is {num_valid_movies}.')

The number of movies with valid financial information is 647.


## **How many movies are there in each of the certification categories (G/PG/PG-13/R)?**

In [27]:
# Count the number of movies in each certification category
certification_counts = data['certification'].value_counts()

print(f'The number of movies in each certification category:\n{certification_counts}')

The number of movies in each certification category:
certification
           918
R          463
PG-13      183
NR          76
PG          63
G           24
NC-17        7
Unrated      1
-            1
Name: count, dtype: int64


## **What is the average revenue per certification category?**

In [28]:
# Calculate the average revenue per certification category
avg_revenue_by_certification = data[data['revenue'] > 0].groupby('certification')['revenue'].mean()

print(f'The average revenue per certification category:\n{avg_revenue_by_certification}')

The average revenue per certification category:
certification
         2.406751e+07
G        1.237463e+08
NR       1.358396e+07
PG       1.218888e+08
PG-13    1.057191e+08
R        4.479793e+07
Name: revenue, dtype: float64


## **What is the average budget per certification category?**

In [29]:
# Calculate the average budget per certification category
avg_budget_by_certification = data[data['budget'] > 0].groupby('certification')['budget'].mean()

print(f'The average budget per certification category:\n{avg_budget_by_certification}')

The average budget per certification category:
certification
         7.450176e+06
G        4.400000e+07
NR       8.271930e+06
PG       4.749992e+07
PG-13    4.507326e+07
R        2.218630e+07
Name: budget, dtype: float64
